In [ ]:
import cv2
import numpy as np

# --- CONFIGURAÇÃO ---
CAMINHO_IMAGEM = "front/2.jpg" 
TOLERANCIA_H = 15  # Tolerância de Cor
TOLERANCIA_S = 60  # Tolerância de Saturação
TOLERANCIA_V = 80  # Tolerância de Brilho

# Define a altura máxima que a janela vai ocupar na tela (em pixels)
# 800 costuma ser bom para telas Full HD (1080p)
ALTURA_MAXIMA_JANELA = 800 

# Variáveis Globais
pontos_poligono = []
fase_selecao_area = True
mask_roi_painel = None
area_total_pixels = 0
img_original = None
img_display = None
mascaras_limpas_acumuladas = None  # Vai guardar tudo que é limpo

def atualiza_calculo():
    global mask_roi_painel, area_total_pixels, mascaras_limpas_acumuladas, img_original, img_display
    
    if mask_roi_painel is None or mascaras_limpas_acumuladas is None: return

    # 1. Lógica Inversa: Tudo que NÃO está na máscara acumulada de "limpos" é sujeira
    mask_sujeira = cv2.bitwise_not(mascaras_limpas_acumuladas)
    
    # 2. Restringe à área do polígono
    mask_final = cv2.bitwise_and(mask_sujeira, mask_roi_painel)
    
    # 3. Limpeza de ruído
    kernel = np.ones((3,3), np.uint8)
    mask_final = cv2.morphologyEx(mask_final, cv2.MORPH_OPEN, kernel)

    # 4. Cálculo
    pixels_sujos = cv2.countNonZero(mask_final)
    porcentagem = 0
    if area_total_pixels > 0:
        porcentagem = (pixels_sujos / area_total_pixels) * 100

    print(f"-> Área Suja Atualizada: {porcentagem:.2f}%")

    # 5. Visualização
    overlay = img_original.copy()
    overlay[mask_final > 0] = (0, 0, 255) # Pinta sujeira de Vermelho
    
    # Reset do display para não acumular desenhos antigos
    img_display = img_original.copy()
    # Redesenha o polígono para referência
    pts = np.array(pontos_poligono, np.int32)
    cv2.polylines(img_display, [pts], True, (0, 255, 255), 2)
    
    # Mistura
    cv2.addWeighted(overlay, 0.6, img_display, 0.4, 0, img_display)
    
    # Texto
    texto = f"Sujeira: {porcentagem:.2f}%"
    cv2.rectangle(img_display, (10, 10), (280, 50), (0,0,0), -1)
    cv2.putText(img_display, texto, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    
    cv2.imshow("Analise Multi-Referencia", img_display)

def mouse_callback(event, x, y, flags, param):
    global pontos_poligono, fase_selecao_area, mask_roi_painel, area_total_pixels, img_display, mascaras_limpas_acumuladas

    if event == cv2.EVENT_LBUTTONDOWN:
        # --- FASE 1: DESENHAR POLÍGONO ---
        if fase_selecao_area:
            pontos_poligono.append((x, y))
            cv2.circle(img_display, (x, y), 5, (0, 255, 0), -1)
            if len(pontos_poligono) > 1:
                cv2.line(img_display, pontos_poligono[-2], pontos_poligono[-1], (0, 255, 0), 2)
            cv2.imshow("Analise Multi-Referencia", img_display)
        
        # --- FASE 2: CLICAR NAS REFERÊNCIAS (FUNDO E GRADE) ---
        else:
            # Pega a cor clicada
            hsv = cv2.cvtColor(img_original, cv2.COLOR_BGR2HSV)
            pixel_hsv = hsv[y, x]
            h, s, v = pixel_hsv
            print(f"\n[NOVA REFERÊNCIA] HSV: {h, s, v}")

            # Cria intervalo dessa cor
            lower = np.array([max(0, int(h)-TOLERANCIA_H), max(0, int(s)-TOLERANCIA_S), max(0, int(v)-TOLERANCIA_V)])
            upper = np.array([min(179, int(h)+TOLERANCIA_H), 255, 255])
            
            # Cria máscara dessa nova cor limpa
            mask_nova_ref = cv2.inRange(hsv, lower, upper)
            
            # Adiciona à memória de "coisas limpas" (OU Lógico)
            if mascaras_limpas_acumuladas is None:
                mascaras_limpas_acumuladas = mask_nova_ref
            else:
                mascaras_limpas_acumuladas = cv2.bitwise_or(mascaras_limpas_acumuladas, mask_nova_ref)
            
            atualiza_calculo()

    elif event == cv2.EVENT_RBUTTONDOWN and fase_selecao_area:
        # Fecha o polígono
        if len(pontos_poligono) > 2:
            fase_selecao_area = False
            pts = np.array(pontos_poligono, np.int32)
            
            mask_roi_painel = np.zeros(img_original.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask_roi_painel, [pts], 255)
            area_total_pixels = cv2.countNonZero(mask_roi_painel)
            
            cv2.polylines(img_display, [pts], True, (0, 255, 255), 2)
            cv2.imshow("Analise Multi-Referencia", img_display)
            print(f"--- ÁREA DEFINIDA ---")
            print(f"1. Clique no FUNDO AZUL do painel.")
            print(f"2. Clique na GRADE CINZA do painel.")
            print(f"3. O resto será considerado sujeira.")

def main():
    global img_original, img_display
    img_original = cv2.imread(CAMINHO_IMAGEM)
    if img_original is None: 
        print(f"Erro ao carregar a imagem: {CAMINHO_IMAGEM}")
        return

    # --- INICIO DA MODIFICAÇÃO DE REDIMENSIONAMENTO ---
    altura_real, largura_real = img_original.shape[:2]
    
    # Calcula a proporção para chegar na altura máxima desejada
    scale = ALTURA_MAXIMA_JANELA / float(altura_real)
    
    # Se a imagem for MENOR que a tela, não precisamos dar zoom (scale = 1), 
    # a menos que você queira aumentar imagens pequenas. 
    # Deixei livre aqui para reduzir imagens grandes.
    if scale < 1.0:
        novo_w = int(largura_real * scale)
        novo_h = int(altura_real * scale)
        img_original = cv2.resize(img_original, (novo_w, novo_h))
    
    print(f"Imagem redimensionada para: {img_original.shape[1]}x{img_original.shape[0]}")
    # --- FIM DA MODIFICAÇÃO ---

    img_display = img_original.copy()

    cv2.namedWindow("Analise Multi-Referencia")
    cv2.setMouseCallback("Analise Multi-Referencia", mouse_callback)

    print("--- INSTRUÇÕES ---")
    print("1. Desenhe a área do painel (Botão Esquerdo -> Direito para fechar).")
    print("2. Clique nas cores que SÃO DO PAINEL (Azul, Cinza, Preto).")
    print("3. Tudo o que você NÃO clicar será pintado de VERMELHO (Sujeira).")
    print("4. 'q' para sair.")

    cv2.imshow("Analise Multi-Referencia", img_original)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'): break
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()